# Module 7: The "Zero to Hero" Deployment Pipeline

This is the final integration. We combine Physics (ZNE) with AI to create a hybrid solver that beats the baseline.

## 7.1 The Hybrid Architecture

1.  **ZNE Layer:** Run at noise scale 1x and 3x to get a linear baseline.
2.  **Feature Extraction:** Extract `[ZNE_Estimate, Depth, CNOT_Count]`.
3.  **AI Correction:** Feed features into the AI to predict the *residual error*.
4.  **Final Result:** `Prediction = ZNE_Estimate + AI_Correction`

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.circuit.random import random_circuit
from qiskit_aer.noise import NoiseModel, thermal_relaxation_error, ReadoutError

# --- 1. SETUP: Mock Noise Environment ---
# We recreate the environment here so this module runs standalone.
def build_noise_model():
    nm = NoiseModel()
    
    # A. 1-Qubit Error (Fast gates)
    error_1q = thermal_relaxation_error(50e-6, 70e-6, 50e-9)
    nm.add_all_qubit_quantum_error(error_1q, ['x', 'h', 'id'])
    
    # B. 2-Qubit Error (Slow gates - Tensor Product)
    error_2q_single = thermal_relaxation_error(50e-6, 70e-6, 400e-9)
    error_2q = error_2q_single.expand(error_2q_single)
    nm.add_all_qubit_quantum_error(error_2q, ['cx'])
    
    return nm

sim_noisy = AerSimulator(noise_model=build_noise_model())

# --- 2. PIPELINE DEFINITION ---
class HackathonPipeline:
    def __init__(self):
        # For the final submission, we use the SVR for stability and speed.
        # (Note: In a real submission, you could load your saved LSTM state dict here)
        self.ai_model = make_pipeline(StandardScaler(), SVR(C=10.0, epsilon=0.01))
        
    def extract_features(self, qc, zne_estimate):
        """
        Feature Vector: [ZNE_Estimate, Circuit_Depth, CNOT_Count]
        """
        ops = qc.count_ops()
        return [zne_estimate, qc.depth(), ops.get('cx', 0)]

    def run_zne(self, qc, simulator):
        """
        Simulates a ZNE run (Baseline).
        For this demo, we run once (1x) and use it as the 'ZNE Estimate'.
        """
        qc_t = transpile(qc, simulator)
        result = simulator.run(qc_t, shots=1000).result()
        counts = result.get_counts()
        
        shots = sum(counts.values())
        p_even = (counts.get('00', 0) + counts.get('11', 0)) / shots
        p_odd = (counts.get('01', 0) + counts.get('10', 0)) / shots
        return p_even - p_odd

    def train(self, training_circuits, ideal_values, simulator):
        print("Training Pipeline...")
        X_train = []
        y_train = []
        
        for qc, ideal in zip(training_circuits, ideal_values):
            # 1. Get Baseline (ZNE)
            zne_est = self.run_zne(qc, simulator)
            # 2. Extract Features
            feats = self.extract_features(qc, zne_est)
            # 3. Target = Residual Error (Ideal - ZNE)
            X_train.append(feats)
            y_train.append(ideal - zne_est)
            
        self.ai_model.fit(X_train, y_train)
        print("Pipeline Trained.")

    def predict(self, target_circuit, simulator):
        # 1. Get Baseline
        zne_est = self.run_zne(target_circuit, simulator)
        # 2. Extract Features
        feats = self.extract_features(target_circuit, zne_est)
        # 3. AI Correction
        correction = self.ai_model.predict([feats])[0]
        return zne_est + correction

# --- 3. EXECUTION ---
# A. Generate Training Data
print("Generating Training Data (Small Batch)...")
train_circuits = [random_circuit(2, d, max_operands=2, measure=True) for d in range(2, 25)]
# Mock Ideal Values (Assuming even depth=1.0, odd=-0.5 for demo speed)
ideal_vals = [1.0 if d % 2 == 0 else -0.5 for d in range(2, 25)]

# B. Train
pipeline = HackathonPipeline()
pipeline.train(train_circuits, ideal_vals, sim_noisy)

# C. Validate on UNSEEN Circuit
test_qc = random_circuit(2, 30, max_operands=2, measure=True) # Deeper than training!
true_val = 1.0 # Assume this is an even-depth identity circuit for testing

raw_result = pipeline.run_zne(test_qc, sim_noisy)
mitigated_result = pipeline.predict(test_qc, sim_noisy)

print(f"\n--- VALIDATION RESULTS ---")
print(f"True Ideal:       {true_val:.4f}")
print(f"Raw (ZNE Only):   {raw_result:.4f}  (Error: {abs(true_val - raw_result):.4f})")
print(f"Hybrid (AI+ZNE):  {mitigated_result:.4f}  (Error: {abs(true_val - mitigated_result):.4f})")

ratio = abs(true_val - raw_result) / abs(true_val - mitigated_result)
print(f"\n🚀 IMPROVEMENT RATIO: {ratio:.2f}x")

if ratio > 1.0:
    print("✅ SUCCESS: Pipeline is beating the baseline!")
else:
    print("❌ ADJUSTMENT NEEDED: More training data required.")